In [4]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import requests
import io
import os # For checking file existence for us-states.json

# Load the dataset
URL_CSV = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"
try:
    response_csv = requests.get(URL_CSV)
    response_csv.raise_for_status() # Raise an exception for HTTP errors
    csv_content = io.StringIO(response_csv.text)
    data = pd.read_csv(csv_content)
    print("Dataset loaded successfully!")
except requests.exceptions.RequestException as e:
    print(f"Error loading main dataset: {e}")
    # Exit or handle gracefully if data cannot be loaded
    data = pd.DataFrame() # Create an empty DataFrame to prevent further errors

# Convert 'Date' column to datetime objects
data['Date'] = pd.to_datetime(data['Date'])

# Extract 'Year' and 'Month' from the Date column
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month

# Get the list of years for the dropdown
year_list = [i for i in range(1980, 2024, 1)]

# --- Start of handling us-states.json for optional Task 1.10 (Choropleth map) ---
geojson_filename = "us-states.json"
URL_GEOJSON = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/us-states.json'

if not os.path.exists(geojson_filename):
    print(f"Downloading {geojson_filename} for optional Task 1.10...")
    try:
        response_geojson = requests.get(URL_GEOJSON)
        response_geojson.raise_for_status()
        with open(geojson_filename, "wb") as f:
            f.write(response_geojson.content)
        print(f"{geojson_filename} downloaded successfully!")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {geojson_filename}: {e}")
        geojson_filename = None # Set to None if download fails
else:
    print(f"{geojson_filename} already exists locally for optional Task 1.10.")
# --- End of handling us-states.json ---


# Initialize the Dash app
app = dash.Dash(__name__)

# REVIEW: Set the app title
# TASK 2.1: Create a Dash application and give it a meaningful title
app.layout = html.Div([
    html.H1("Automobile Sales Statistics Dashboard",
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 24}),
    html.Div([
        # TASK 2.2: Add drop-down menus to your dashboard with appropriate titles and options
        html.Label("Select Report Type:"),
        dcc.Dropdown(
            id='dropdown-statistics',
            options=[
                {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
            ],
            placeholder='Select a report type',
            value='Select Statistics', # Default value
            style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
        )
    ]),
    html.Div([
        html.Label("Select Year:"),
        dcc.Dropdown(
            id='select-year',
            options=[{'label': i, 'value': i} for i in year_list],
            placeholder='Select Year',
            value='Select-year', # Default value
            style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
        )
    ]),
    # TASK 2.3: Add a division for output display with appropriate id and classname property
    html.Div([
        html.Div(id='output-container', className='chart-grid', style={'display': 'flex', 'flex-wrap': 'wrap'})
    ])
])

# TASK 2.4: Creating Callbacks; Define the callback function to update the input container based on the selected statistics and the output container

# Callback to enable/disable year dropdown based on report type
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics', component_property='value')
)
def update_input_container(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False
    else:
        return True

# Callback to update output container with plots
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value'),
     Input(component_id='select-year', component_property='value')]
)
def update_output_container(selected_statistics, input_year):
    if selected_statistics == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]

        # TASK 2.5: Create and display graphs for Recession Report Statistics
        # Plot 1: Automobile sales fluctuate over Recession Period (year-wise) using line chart
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec,
                x='Year',
                y='Automobile_Sales',
                title="Average Automobile Sales Fluctuation Over Recession Period (Year-wise)"))

        # Plot 2: Average number of vehicles sold by vehicle type and represent as a Bar chart
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure=px.bar(average_sales,
            x='Vehicle_Type',
            y='Automobile_Sales',
            title="Average Number of Vehicles Sold by Vehicle Type During Recessions"))

        # Plot 3: Pie chart for total expenditure share by vehicle type during recessions
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec,
            values='Advertising_Expenditure',
            names='Vehicle_Type',
            title="Total Expenditure Share by Vehicle Type During Recessions"
            )
        )

        # Plot 4: Bar chart for the effect of unemployment rate on vehicle type and sales
        unemp_data = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(figure=px.bar(unemp_data,
        x='unemployment_rate',
        y='Automobile_Sales',
        color='Vehicle_Type',
        labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
        title='Effect of Unemployment Rate on Vehicle Type and Sales'))

        return [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1), html.Div(children=R_chart2)], style={'display': 'flex', 'flex-direction': 'row', 'flex-wrap': 'wrap'}),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3), html.Div(children=R_chart4)], style={'display': 'flex', 'flex-direction': 'row', 'flex-wrap': 'wrap'})
        ]

    elif (input_year and selected_statistics == 'Yearly Statistics'): # Check for Yearly Statistics.
        yearly_data = data[data['Year'] == input_year]

        # TASK 2.6: Create and display graphs for Yearly Report Statistics
        # Plot 1: Yearly Automobile sales using line chart for the whole period.
        yas = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yas,
                                            x='Year',
                                            y='Automobile_Sales',
                                            title='Yearly Automobile Sales (Overall Period)'))

        # Plot 2: Total Monthly Automobile sales using line chart for the selected year.
        mas = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(mas,
                                            x='Month',
                                            y='Automobile_Sales',
                                            title=f'Total Monthly Automobile Sales in {input_year}'))

        # Plot 3: Bar chart for average number of vehicles sold during the given year
        avr_vdata = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(figure=px.bar(avr_vdata,
                                           x='Vehicle_Type',
                                           y='Automobile_Sales',
                                           title=f'Average Vehicles Sold by Vehicle Type in {input_year}'))

        # Plot 4: Total Advertisement Expenditure for each vehicle using pie chart
        exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(exp_data,
            values='Advertising_Expenditure',
            names='Vehicle_Type',
            title=f'Total Advertisement Expenditure for Each Vehicle in {input_year}')
        )

        return [
            html.Div(className='chart-item', children=[html.Div(children=Y_chart1), html.Div(children=Y_chart2)], style={'display': 'flex', 'flex-direction': 'row', 'flex-wrap': 'wrap'}),
            html.Div(className='chart-item', children=[html.Div(children=Y_chart3), html.Div(children=Y_chart4)], style={'display': 'flex', 'flex-direction': 'row', 'flex-wrap': 'wrap'})
        ]
    else:
        return None # No plots to display initially or if selection is invalid

if __name__ == '__main__':
    app.run(debug=True) # Set debug=True for development to auto-reload

Dataset loaded successfully!
us-states.json already exists locally for optional Task 1.10.


In [2]:
import sys
print(sys.executable)
print(sys.version)

c:\Users\maste\AppData\Local\Programs\Python\Python313\python.exe
3.13.5 (tags/v3.13.5:6cb20a2, Jun 11 2025, 16:15:46) [MSC v.1943 64 bit (AMD64)]
